In [1]:
import pandas as pd
import numpy as np
import torch
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data as Data
import lmdb

from torch.autograd import Variable
from warpctc_pytorch import CTCLoss
import sys

from torch.utils.data import Dataset
import torchvision.transforms as transforms
import six
import sys
from PIL import Image

In [2]:
sys.path.insert(1,'./crnn.pytorch')

In [3]:
import models.crnn as crnn
import dataset

In [4]:
class strLabelConverter(object):
    """Convert between str and label.
    NOTE:
        Insert `blank` to the alphabet for CTC.
    Args:
        alphabet (str): set of the possible characters.
    """

    def __init__(self, alphabet, ignore_case=True):
        self._ignore_case = ignore_case
        if self._ignore_case:
            alphabet = alphabet.lower()
        self.alphabet = alphabet + '-'  # for `-1` index

        self.dict = {}
        for i, char in enumerate(alphabet):
            # NOTE: 0 is reserved for 'blank' required by wrap_ctc
            self.dict[char] = i + 1

In [5]:
train_dataset = dataset.lmdbDataset(root='/image/Chen/lmdb/')

In [10]:
batchSize = 64

In [11]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batchSize,shuffle=True,num_workers=4,
                                           collate_fn=dataset.alignCollate(imgH=32, imgW=320))

In [13]:
alphabet = '零壹貳參肆伍陸柒捌玖拾佰仟萬億兆元整'

In [14]:
nclass = len(alphabet) + 1
nc = 1
criterion = CTCLoss()

In [ ]:
converter = utils.strLabelConverter(alphabet)

In [ ]:
alphabet = alphabet + '-'  # for `-1` index
alphabet_dict = {}
for i, char in enumerate(alphabet):
    # NOTE: 0 is reserved for 'blank' required by wrap_ctc
    alphabet_dict[char] = i + 1

In [17]:
model = crnn.CRNN(32, 1, 19, 256)
model = model.cuda()
model.load_state_dict(torch.load("./crnn.pytorch/data/crnnV1.pkl"))

RuntimeError: CUDA error: all CUDA-capable devices are busy or unavailable

In [ ]:
image = torch.FloatTensor(batchSize, 1, 32, 32)
text = torch.IntTensor(batchSize * 5)
length = torch.IntTensor(batchSize)

In [ ]:
image = image.cuda()
criterion = criterion.cuda()

In [ ]:
model = torch.nn.DataParallel(model)
image = image.cuda()
criterion = criterion.cuda()

In [ ]:
image = Variable(image)
text = Variable(text)
length = Variable(length)

In [ ]:
optimizer = optim.Adam(model.parameters(), lr=0.01126,betas=(0.01, 0.999))

In [ ]:
for epoch in range(1000):
    train_iter = iter(train_loader)
    i = 0
    while i < len(train_loader):
        for p in model.parameters():
            p.requires_grad = True
        model.train()

        i += 1

        if i % 10 == 0:
            print('[iteration%d/%d][%d/%d] ' %
                  (epoch, 1000, i, len(train_loader), ))

In [ ]:
torch.save(model.state_dict(), './crnn.pytorch/data/crnnV2.pkl')

In [ ]:
model

In [ ]:
from PIL import Image
Image.open('./crnn.pytorch/data/GF10222057890039.jpg')
